# Cleaning Folha Dataset

In [1]:
import pandas as pd
import numpy as np
import html
import re
from datasets import Dataset, DatasetDict, load_dataset

pd.set_option('display.max_rows', 120)

/Users/luokamura/miniconda3/envs/scrapy-folha/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load Dataset and Overview

In [2]:
dataset_filepath = "../data/folha_2013_2023_raw.zip"
folha = pd.read_csv(dataset_filepath)
folha.head()

,title,text,date,category,subcategory,link
0,Feira de produtos e cultura ligada à maconha a...,Estandes produtores e importadores de óleos de...,2023-09-18 10:08:00,cotidiano,NaN,https://www1.folha.uol.com.br/cotidiano/2023/0...
1,Após Luva de Pedreiro indicar que pagaria pens...,"Na tarde de domingo (17), Luva de Pedreiro abr...",2023-09-18 08:40:00,celebridades,NaN,https://f5.folha.uol.com.br/celebridades/2023/...
2,Sabesp quer automatizar saneamento em todo o e...,A Sabesp está prestes a concluir um projeto-pi...,2023-09-18 10:00:00,mercado,NaN,https://www1.folha.uol.com.br/mercado/2023/09/...
3,Sons de S.Paulo: poluição sonora em Embu das A...,Sons de S.Paulo é uma série do Música em Letra...,2023-09-18 09:42:00,blogs,musica-em-letras,https://www1.folha.uol.com.br/blogs/musica-em-...
4,OMS cobra da China 'acesso total' para investi...,O diretor-geral da OMS (Organização Mundial da...,2023-09-18 09:49:00,equilibrioesaude,NaN,https://www1.folha.uol.com.br/equilibrioesaude...


In [3]:
# number of rows and columns
print(f"Rows: {folha.shape[0]}")
print(f"Columns: {folha.shape[1]}")

Rows: 1081495
Columns: 6


## 2. Checking columns

### 2.1. title

In [5]:
# NaN titles
print(f"NaN title rows: {folha['title'].isna().sum()}")
print(f"% NaN title rows: {folha['title'].isna().sum() / len(folha) * 100} %")

NaN title rows: 0
% NaN title rows: 0.0 %


In [6]:
# check for double spaces, tabs or line breaks
cond = folha['title'].apply(lambda x: str(x)).str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'title']

356778     Morcheeba retorna ao Brasil para  Nublu Jazz F...
360086     ​    Para leitores, Carnaval de São Paulo sobr...
392740     Roberta Miranda reúne novatas em CD para mostr...
471964               Drops Olímpicos - Brasil  4X0 Dinamarca
480160     Vídeo mostra o momento em  que um dos atirador...
                                 ...                        
1034432    Parlamento grego aprova 2º pacote de leis para...
1034809    Grécia leva ao Parlamento novo pacote de ajust...
1035005    Grécia inicia o pagamento de  6,25 bilhões ao ...
1035984    Comissão da UE propõe empréstimo de curto praz...
1055341    Moradores esperam por obras públicas  na 'Ilha...
Name: title, Length: 3936, dtype: object

In [7]:
# check for double spaces, tabs or line breaks
folha.loc[cond, 'title'].tail(1).values[0]

"Moradores esperam por obras públicas  na 'Ilha da Paciência'"

### 2.2. text

In [8]:
# NaN text
print(f"NaN text rows: {folha['text'].isna().sum()}")
print(f"% NaN text rows: {folha['text'].isna().sum() / len(folha) * 100} %")

NaN text rows: 504760
% NaN text rows: 46.67243029325147 %


In [9]:
# check for double spaces, tabs or line breaks
cond = folha['text'].apply(lambda x: str(x)).str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'text'].tail(1).values[0]

'DE SÃO PAULOVeja atrações culturais para curtir em São Paulo nesta quinta (5).Cirque du SoleilA companhia canadense Cirque\xa0 du\xa0 Soleil –que volta ao Brasil depois de quatro anos–\xa0apresenta o espetáculo \'Amaluna\', o\xa0primeiro com elenco majoritariamente feminino (65% dos atores são mulheres). Após passar por dez países, a turnê chega ao parque Villa-Lobos nesta quinta (5). Saiba mais sobre o espetáculo.Pq. Villa-Lobos - Av. Prof. Fonseca Rodrigues, 2.001, Alto de Pinheiros. 2.482 lugares. Ter. a sex.: 21h. Sáb.: 17h30 e 21h. Dom.: 16h e 19h30. Estreia 5/10. Até 17/12. Ingr.: R$ 250 a R$ 450 p/ tudus.com.br.                                       Mais                              \'Why the Horse?\'\xa0Texto: Fábio Furtado. Direção: Maria Alice Vergueiro. Com: Carolina Splendore, Robson Catalunha e Otávio Ortega. 50 min. 16 anos.\xa0A atriz Maria Alice Vergueiro encena pela centésima vez essa peça, que também dirige, em curta temporada no Teatro Oficina. Instigada pelo tema d

In [43]:
text = folha.loc[cond, 'text'].tail(10).values[0]
test = re.sub(r"\s+", " ", text)
test

'Tite escolheu o amistoso desta sexta (10) para testar os reservas para a Copa, mas o adversário acabou não ajudando. Com facilidade, o time misto do Brasil venceu o Japão, por 3 a 1, em Lille, no penúltimo amistoso do ano. Todos os gols da seleção foram marcados na fase inicial. O primeiro acabou contando com a ajuda do árbitro de vídeo, que apontou um pênalti sofrido por Fernandinho após o juiz pedir o auxílio da tecnologia. Com a falta confirmada, Neymar aproveitou a oportunidade e fez o seu oitavo gol na carreira contra os japoneses. Marcelo e Gabriel Jesus completaram a vitória. O gol dos japoneses foi marcado aos 17min do segundo tempo por Makino, que ganhou na cabeça de Jemerson. Com a vitória, a seleção manteve a invencibilidade diante do Japão. Nesta terça (10), o Brasil fará o último amistoso deste ano. Com todos os titulares, a seleção enfrentará a Inglaterra, em Wembley. No próximo ano, o treinador terá apenas dois jogos antes de decidir a lista dos convocados para a disput

### 2.3. date

In [10]:
# NaN data
print(f"NaN date rows: {folha['date'].isna().sum()}")
print(f"% NaN date rows: {folha['date'].isna().sum() / len(folha) * 100} %")

NaN date rows: 503321
% NaN date rows: 46.539373737280336 %


### 2.4. category

In [11]:
# NaN categories
print(f"NaN category rows: {folha['category'].isna().sum()}")
print(f"% NaN category rows: {folha['category'].isna().sum() / len(folha) * 100} ")

NaN category rows: 0
% NaN category rows: 0.0 


In [12]:
len(folha['category'].value_counts().sort_values(ascending = False))

120

In [13]:
# number of rows per category
folha['category'].value_counts().sort_values(ascending = False)

category
mercado                                            205893
colunas                                            129648
poder                                               95776
cotidiano                                           92004
esporte                                             83282
mundo                                               81757
ilustrada                                           73719
fsp                                                 35414
celebridades                                        32565
televisao                                           23787
opiniao                                             22806
paineldoleitor                                      16391
equilibrioesaude                                    15093
saopaulo                                            12349
tec                                                 10803
educacao                                             9113
colunistas                                           8610
inter

In [14]:
# colunas x colunistas (seems to be the same thing)
print(folha.query("category == 'colunas'").iloc[2, -1])
print(folha.query("category == 'colunistas'").iloc[0, -1])

https://www1.folha.uol.com.br/colunas/painelsa/2023/09/construtoras-chinesas-farao-uma-das-maiores-pontes-do-pais.shtml
https://f5.folha.uol.com.br/colunistas/de-faixa-a-coroa/2023/09/mister-international-2023-brasileiro-fica-em-3o-e-tailandes-vence.shtml


In [15]:
# restaurantes x comidas (seems to be the same thing)
print(folha.query("category == 'restaurantes'").iloc[0, -1])
print(folha.query("category == 'comida'").iloc[1, -1])

https://guia.folha.uol.com.br/restaurantes/2023/09/festivais-fazem-degustacao-gratis-de-comida-e-bebida-coreanas-em-sp-veja-como-participar.shtml
https://www1.folha.uol.com.br/comida/2023/09/wine-tour-2023-tera-ator-juan-alba-e-degustacao-de-100-vinhos-em-sp.shtml


In [16]:
# cinemas-e-series x cinema x 42a-mostra-internacional-de-cinema x 41a-mostra-internacional-de-cinema (seems to be the same thing)
print(folha.query("category == 'cinema-e-series'").iloc[2, -1])
print(folha.query("category == 'cinema'").iloc[4, -1])
print(folha.query("category == '42a-mostra-internacional-de-cinema'").iloc[0, -1])
print(folha.query("category == '41a-mostra-internacional-de-cinema'").iloc[1, -1])

https://f5.folha.uol.com.br/cinema-e-series/2023/09/amazon-vai-estrear-serie-inspirada-do-filme-sr-e-sra-smith-em-2024-veja-detalhes.shtml
https://guia.folha.uol.com.br/cinema/2023/08/a-chamada-com-liam-neeson-e-gran-turismo-estreiam-nos-cinemas-de-sao-paulo.shtml
https://guia.folha.uol.com.br/42a-mostra-internacional-de-cinema/2018/10/vencedor-do-leao-de-ouro-e-documentario-sobre-eleitores-de-trump-sao-destaque-do-dia-na-mostra.shtml
https://guia.folha.uol.com.br/41a-mostra-internacional-de-cinema/2017/11/veja-tres-filmes-em-destaque-no-ultimo-dia-da-mostra-de-cinema.shtml


In [17]:
# sao-paulo x saopaulo x o-melhor-de-sao-paulo (seems to be the same thing)
print(folha.query("category == 'sao-paulo'").iloc[9, -1])
print(folha.query("category == 'saopaulo'").iloc[30, -1])
print(folha.query("category == 'o-melhor-de-sao-paulo'").iloc[12, -1])

https://agora.folha.uol.com.br/sao-paulo/2021/11/arvore-de-natal-na-ponte-estaiada-em-sp-sera-inaugurada-neste-sabado-27.shtml
https://www1.folha.uol.com.br/saopaulo/viaja-sp/melhores/2019/05/1987966-cvc-melhor-marca-de-turismo-e-agencia-de-viagem-online-investe-em-parcerias-com-grandes-eventos-como-o-rock-in-rio.shtml
https://www1.folha.uol.com.br/o-melhor-de-sao-paulo/2023/08/heinz-e-a-marca-preferida-de-ketchup-e-mostarda-dos-paulistanos.shtml


## 3. Clean Dataset

In [18]:
folha_filtered = folha.copy()

In [19]:
def group_categories(df):
    """
    Group similar categories into one.
    """
    df.loc[df['category'] == 'equilibrioesaude', 'category'] = 'equilibrio'
    df.loc[df['category'] == 'tv', 'category'] = 'televisao'
    df.loc[df['category'] == 'mundo', 'category'] = 'internacional'
    df.loc[df['category'] == 'cinema', 'category'] = 'cinema-e-series'
    df.loc[df['category'] == 'equilibrio', 'category'] = 'equilibrio-e-saude'

    return df

In [20]:
def clean_text(df, column):
    """
    Cleans text column, removing tabs, line breaks, double spaces, unicode and html strings. Removes NaN values.
    """

    # remove NaN values
    df.dropna(subset = column, inplace = True)

    # remove duplicated column values
    df.drop_duplicates(subset=column, inplace=True)

    # remove tabs and line breaks
    df[column] = df[column].str.replace(r"\t+|\n+", "")

    # remove double spaces and strip
    df[column] = df[column].str.strip() \
                        .apply(lambda text: re.sub(r"\s+", " ", text))

    # remove html strings
    df[column] = df[column].apply(lambda text: html.unescape(text))

    return df

### 3.1. Filter categories

In [21]:
categories_to_drop = [
    "folhinha",
    "webstories",
    "seminariosfolha",
    "banco-de-dados",
    "sobretudo",
    "voceviu",
    "ilustrissima",
    "blogs",
    "colunistas",
    "sao-paulo",
    "saopaulo",
    "paineldoleitor",
    "opiniao",
    "ilustrada",
    "colunas",
    "fsp",
    "bbc",
    "passeios",
    "shows",
    "restaurantes",
    "bichos",
    "humanos",
    "teatro",
    "multimidia",
    "serafina",
    "asmais"
]

In [22]:
aux = pd.DataFrame(folha['category'].value_counts().rename('qtd').sort_values(ascending=False))
aux = aux.query("qtd > 1000").reset_index()
aux = aux.query("~category.isin(@categories_to_drop)")
filtered_categories = aux['category'].unique()
print(len(filtered_categories))
print(filtered_categories)

23
['mercado' 'poder' 'cotidiano' 'esporte' 'mundo' 'celebridades'
 'televisao' 'equilibrioesaude' 'tec' 'educacao' 'internacional' 'turismo'
 'ciencia' 'ambiente' 'tv' 'comida' 'grana' 'empreendedorsocial'
 'cinema-e-series' 'musica' 'cinema' 'podcasts' 'equilibrio']


In [23]:
print(f"Before filtering categories: {len(folha_filtered)}")
folha_filtered = folha_filtered.query("category.isin(@filtered_categories)").copy()
print(f"After filtering categories: {len(folha_filtered)}")

Before filtering categories: 1081495
After filtering categories: 707439


### 3.2. Clean text

In [24]:
def clean_dataset(df):
    """
    Pipeline to clean dataset.
    """
    print(f"Rows in dataset: {len(df)}")
    # cleans title
    df = clean_text(df, 'title')
    print(f"Rows in dataset after title cleaning: {len(df)}")

    # cleans text
    df = clean_text(df, 'text')
    print(f"Rows in dataset after text cleaning: {len(df)}")

    # group categories
    df = group_categories(df)
    print(f"Final categories: {len(df['category'].unique())}")

    return df

In [25]:
# clean and export
folha_filtered = clean_dataset(folha_filtered)

Rows in dataset: 707439
Rows in dataset after title cleaning: 707439
Rows in dataset after text cleaning: 354823
Final categories: 19


### 3.3. Filter texts with less than X characters

In [26]:
print(folha_filtered.shape)
folha_filtered = folha_filtered.loc[folha_filtered['text'].str.len() > 300]
print(folha_filtered.shape)

(354823, 6)
(352228, 6)


### 3.4. Filter Columns

In [27]:
folha_filtered = folha_filtered.loc[:, ["title", "text", "date", "category", "link"]]
folha_filtered.head()

,title,text,date,category,link
0,Feira de produtos e cultura ligada à maconha a...,Estandes produtores e importadores de óleos de...,2023-09-18 10:08:00,cotidiano,https://www1.folha.uol.com.br/cotidiano/2023/0...
1,Após Luva de Pedreiro indicar que pagaria pens...,"Na tarde de domingo (17), Luva de Pedreiro abr...",2023-09-18 08:40:00,celebridades,https://f5.folha.uol.com.br/celebridades/2023/...
2,Sabesp quer automatizar saneamento em todo o e...,A Sabesp está prestes a concluir um projeto-pi...,2023-09-18 10:00:00,mercado,https://www1.folha.uol.com.br/mercado/2023/09/...
4,OMS cobra da China 'acesso total' para investi...,O diretor-geral da OMS (Organização Mundial da...,2023-09-18 09:49:00,equilibrio-e-saude,https://www1.folha.uol.com.br/equilibrioesaude...
6,Agência de mineração mantém ativos processos e...,"Pesquisa da ONG Ekō, que monitora o comportame...",2023-09-18 10:00:00,ambiente,https://www1.folha.uol.com.br/ambiente/2023/09...


### 3.5. Check changes

In [28]:
def calculate_metrics(df):
    # calculate descriptive statistics
    metrics = df.groupby('category')['text'] \
                .apply(lambda x: x.str.len().describe()) \
                .reset_index()
    metrics = pd.pivot_table(
        metrics,
        values = "text",
        index = "category",
        columns = "level_1"
        ) \
        .add_suffix("_len_text") \
        .reset_index()
    
    # calculate percentage
    metrics.rename(columns = {'count_len_text' : 'count'}, inplace = True)
    pct = pd.DataFrame(df['category'].value_counts(normalize = True))

    # calculate mean title length
    mean_len_title = df.groupby('category')['title'] \
                       .apply(lambda x: np.mean(x.str.len())) \
                       .rename("mean_len_title") \
                       .round() \
                       .astype(int)

    # join metrics tables
    pct_mean_len_title = pct.join(mean_len_title)
    metrics = metrics.merge(
        pct_mean_len_title,
        left_on = "category",
        right_index = True
        )
    metrics['count'] = metrics['count'].astype(int)
    metrics = metrics[
        [
            "category", "count", "proportion", "mean_len_title", "25%_len_text", 
            "50%_len_text", "75%_len_text", "max_len_text", 
            "mean_len_text", "min_len_text", "std_len_text"
        ]
    ].sort_values(by = "count", ascending = False)

    return metrics

In [29]:
metrics = calculate_metrics(folha_filtered)
metrics

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,62139,69,2059.00,3130.0,4567.00,102232.0,3611.080931,301.0,2344.350642,0.176417
11,internacional,51617,71,1686.00,2868.0,4485.00,50032.0,3437.321929,302.0,2409.965613,0.146544
15,poder,51288,72,2280.00,3409.0,5192.00,215128.0,4039.852305,301.0,2995.367889,0.145610
5,cotidiano,40392,70,1881.75,2865.0,4313.25,37019.0,3320.399163,301.0,2011.257750,0.114676
9,esporte,35327,67,1448.00,2279.0,3418.50,57574.0,2721.151131,301.0,1885.601996,0.100296
1,celebridades,32261,74,894.00,1337.0,1862.00,20609.0,1576.299960,301.0,1202.699372,0.091591
17,televisao,25825,73,732.00,1243.0,2043.00,34127.0,1632.145944,301.0,1415.961818,0.073319
8,equilibrio-e-saude,13690,74,2759.00,4210.5,5914.75,31998.0,4687.155734,305.0,2799.590691,0.038867
3,cinema-e-series,6100,76,1273.00,1956.0,3559.75,52737.0,3081.770328,312.0,3513.115712,0.017318
6,educacao,5150,70,2468.25,3629.5,5020.75,33713.0,3939.154369,302.0,2083.177998,0.014621


## 4. Export and push to hub

### 4.1. Export to CSV (.zip)

In [36]:
# Export file
filename = 'folha_2015_2023_clean_cats'
print(f"Exporting file {filename}")
compression_options = dict(method='zip', archive_name=f'{filename}.csv')
folha_filtered.to_csv(f'../data/{filename}.zip', compression=compression_options, index = False)

Exporting file folha_2015_2023_clean_cats


### 4.2. Export to Hub

In [28]:
# dataset_filepath = "../data/folha_2013_2023_clean_cats.zip"
# folha_filtered = pd.read_csv(dataset_filepath)
# print(folha_filtered.columns)
# print(folha_filtered.shape)

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')
(354534, 6)


In [40]:
rename_dict = {
    'mercado':'Mercado',
    'poder': 'Poder',
    'internacional': 'Internacional',
    'cotidiano': 'Cotidiano',
    'esporte': 'Esporte',
    'celebridades': 'Celebridades',
    'televisao': 'Televisão',
    'equilibrio-e-saude': 'Equilibrio e Saúde',
    'cinema-e-series': 'Cinema e Séries',
    'educacao': 'Educação',
    'grana': 'Grana ou Dinheiro',
    'ambiente': 'Meio Ambiente',
    'tec': 'Tecnologia',
    'ciencia': 'Ciência',
    'turismo': 'Turismo',
    'musica': 'Música',
    'empreendedorsocial': 'Empreendedorismo Social',
    'podcasts': 'Podcasts',
    'comida': 'Comida'
}

folha_filtered['category_natural_language'] = folha_filtered['category'].map(rename_dict)
folha_filtered['category_natural_language'].unique()

array(['Cotidiano', 'Celebridades', 'Mercado', 'Equilibrio e Saúde',
       'Meio Ambiente', 'Poder', 'Internacional', 'Comida', 'Televisão',
       'Podcasts', 'Esporte', 'Educação', 'Música', 'Ciência',
       'Tecnologia', 'Cinema e Séries', 'Turismo',
       'Empreendedorismo Social', 'Grana ou Dinheiro'], dtype=object)

In [41]:
test = folha_filtered.sample(frac = 0.5, random_state = 42)
metrics_test = calculate_metrics(test)
metrics_test

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,30918,69,2069.25,3132.0,4568.00,102232.0,3614.874410,301.0,2364.980156,0.175557
11,internacional,25816,71,1684.00,2864.5,4473.00,48411.0,3430.901418,302.0,2399.574559,0.146587
15,poder,25758,72,2265.00,3389.0,5171.00,215128.0,4018.773585,301.0,3064.160725,0.146258
5,cotidiano,20162,70,1880.25,2873.0,4331.00,37019.0,3334.861621,303.0,2041.278136,0.114483
9,esporte,17779,67,1457.00,2279.0,3438.00,29714.0,2729.847967,303.0,1846.463359,0.100952
1,celebridades,16292,74,893.00,1335.5,1863.25,17741.0,1566.447520,301.0,1169.894067,0.092508
17,televisao,12881,73,738.00,1247.0,2050.00,22681.0,1631.301374,301.0,1389.901288,0.073140
8,equilibrio-e-saude,6729,74,2768.00,4236.0,5971.00,27150.0,4694.882449,331.0,2756.760059,0.038208
3,cinema-e-series,3035,76,1278.00,1957.0,3596.00,52575.0,3088.909720,428.0,3492.994978,0.017233
6,educacao,2569,70,2482.00,3624.0,5051.00,22681.0,3923.881666,365.0,2044.219652,0.014587


In [42]:
train = folha_filtered.loc[~folha_filtered.index.isin(test.index)]
metrics_train = calculate_metrics(train)
metrics_train

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,31221,69,2046.00,3125.0,4564.00,60816.0,3607.324269,302.0,2323.772641,0.177277
11,internacional,25801,71,1687.00,2877.0,4500.00,50032.0,3443.746173,303.0,2420.347517,0.146502
15,poder,25530,72,2293.00,3428.0,5212.75,136497.0,4061.119271,301.0,2924.227175,0.144963
5,cotidiano,20230,70,1883.00,2861.0,4296.00,32006.0,3305.985319,301.0,1980.831071,0.114869
9,esporte,17548,67,1437.00,2280.0,3399.50,57574.0,2712.339811,301.0,1924.456837,0.099640
1,celebridades,15969,74,895.00,1338.0,1860.00,20609.0,1586.351681,301.0,1235.226437,0.090674
17,televisao,12944,73,723.00,1240.0,2035.25,34127.0,1632.986403,301.0,1441.481085,0.073498
8,equilibrio-e-saude,6961,74,2754.00,4183.0,5859.00,31998.0,4679.686539,305.0,2840.558027,0.039526
3,cinema-e-series,3065,75,1266.00,1953.0,3527.00,52737.0,3074.700816,312.0,3533.482338,0.017404
6,educacao,2581,70,2455.00,3643.0,4971.00,33713.0,3954.356064,302.0,2121.531959,0.014655


In [43]:
train_dataset = Dataset.from_pandas(train[["title", "text", "date", "category", "category_natural_language", "link"]])
test_dataset = Dataset.from_pandas(test[["title", "text", "date", "category", "category_natural_language", "link"]])
train_dataset = train_dataset.remove_columns("__index_level_0__")
test_dataset = test_dataset.remove_columns("__index_level_0__")

In [44]:
folha_dataset = DatasetDict(
    {
        'train': train_dataset,
        'test': test_dataset 
    }
)

In [45]:
print(folha_dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'date', 'category', 'category_natural_language', 'link'],
        num_rows: 176114
    })
    test: Dataset({
        features: ['title', 'text', 'date', 'category', 'category_natural_language', 'link'],
        num_rows: 176114
    })
})


In [47]:
# folha_dataset.push_to_hub("iara-project/news-articles-ptbr-dataset")

Deleting unused files from dataset repository: 100%|██████████| 2/2 [00:00<00:00,  3.83it/s]
